In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta

In [5]:
## Pitchers url: 'https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=1&season=2023&season1=2023&ind=0&pageitems=2000000000&qual=0&team=0&startdate=' + start_date + '&enddate=' + temp_date'
## Batters url: 'https://www.fangraphs.com/leaders/major-league?pos=all&stats=bat&lg=all&type=1&season=2023&month=1000&season1=2023&ind=0&pageitems=2000000000&qual=0&team=0&startdate=' + start_date + '&enddate=' + temp_date'

In [6]:
## To simplify webscraping I'm going to loop through each day of the season, pull all "Advanced Stats" 
## for all players for each day of the 2023 season. I'll then do the same for pitchers. Once I have all
## that data I'll worry about creating a clean data set for training a model

In [7]:
## Using options while getting the url to avoid issues with grabbing url and saving html
options = webdriver.ChromeOptions() # Adding specifications on the driver to avoid timeouts loading the url
options.add_argument('--headless')  # Keeps from opening a chrome tab 
options.add_argument('--pageLoadStrategy=none') # Including manual arguments and waiting inputs for the page to load
options.add_argument("--blink-settings=imagesEnabled=false")  # Disable images
options.add_argument("--autoplay-policy=no-user-gesture-required")  # Disable video autoplay
options.add_argument("--disable-notifications") # Avoiding one of Fangraphs pop ups

In [8]:
## Function for getting the html from a url for hitters stats between a start date and end date
def get_html(url):
    url = url
    driver = webdriver.Chrome(options = options)
    try:
        driver.set_page_load_timeout(30)  # Set page load timeout
        driver.get(url)
        time.sleep(3)
        html = driver.page_source
        # Use explicit wait for a specific element
        #wait = WebDriverWait(driver, 30)
        #wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'table-scroll')))
    except Exception as e:
        print("Exception occured: ", e)
    
    driver.quit()
    return html

In [9]:
## Parse the webpage html and extract the html for just the specific table
def table_parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    html_table = soup.find('div', class_='table-scroll').find('table')
    return(html_table)

In [10]:
## convert table html into a dataframe
def html_converter(html_table):
    ## Create the header for the dataframe
    head = html_table.find_all('th') 
    header = [cell.get_text(strip=True) for cell in head]
    
    ## HTML has hidden text in it that we need to get rid of
    updated_header = []
    for head in header:
        index = head.find(' ')
        if index == -1:
            updated_header.append(head)
        else:
            i = int(index/2)
            updated_header.append(head[:i])

    df = pd.DataFrame(columns=updated_header) ## Create dataframe with above header
    
    
    ## Append the rows to the dataframe
    rows = html_table.find_all('tr')[1:] ## Getting the rows (skipping the header)
    for row in rows:
        values = row.find_all('td')   
        new_row = [value.get_text(strip=True) for value in values]
        df.loc[len(df)] = new_row # Append new row to the bottom of the dataframe
    
    return df
        
   

In [11]:
## Formula for increasing the date by one day for the url
def next_day(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    add_day = datetime_object + timedelta(days=1)
    next_day = add_day.strftime("%Y-%m-%d")
    return next_day

In [12]:
# Iterate through all days in the year and collect hitting or pitching data. Dates entered in format "YYYY-MM-DD"
# and positions are "Batter" or "Pitcher"

def collector(start_date, end_date, position):
    temp_date = next_day(start_date)
    i = 0
    daily_data = pd.DataFrame()
    
    
    
    ## Iterates through each day of the season, grabs the data and adds it to the final dataframe
    while temp_date<end_date:
                ## Define the url to webscrape from based on the position
        if position == 'Hitter':
            url = 'https://www.fangraphs.com/leaders/major-league?pos=all&stats=bat&lg=all&type=1&season=2023&month=1000&season1=2023&ind=0&pageitems=2000000000&qual=0&team=0&startdate=' + start_date + '&enddate=' + temp_date
        elif position == 'Pitcher':
            url = 'https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=1&ind=0&pageitems=2000000000&qual=0&team=0&startdate=' + start_date + '&enddate=' + temp_date + '&month=1000&season1=2022&season=2022&sortcol=3&sortdir=desc&pagenum=1'
        else:
            print("Invalid position type: " +  position + " entered, Hitter or Pitcher expected")
        try:
            html = get_html(url)
            html_table = table_parse(html)
            data_to_date = html_converter(html_table)
            if i==0:
                daily_data = data_to_date
                daily_data['Stats Through This Date'] = temp_date
                i+=1
            else:
                data_to_date['Stats Through This Date'] = temp_date
                daily_data = pd.concat([daily_data, data_to_date], ignore_index=True)
                i+=1
            temp_date = next_day(temp_date)
            print(temp_date)
            print(url)
            display(daily_data.head(2))
            display(daily_data.tail(2))
        except:
            temp_date = next_day(temp_date) 
            print(temp_date)
    return daily_data

In [13]:
position = "Pitcher"

In [14]:
pitcher_data = collector("2022-04-06", "2022-10-03", position)

2022-04-08
https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=1&ind=0&pageitems=2000000000&qual=0&team=0&startdate=2022-04-06&enddate=2022-04-07&month=1000&season1=2022&season=2022&sortcol=3&sortdir=desc&pagenum=1


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
0,1,Josh Staumont,KCR,0.00,27.00,0.00,0.00,,0.0%,50.0%,...,312,310,,0.00,12.11,-12.11,,12.11,16.56,2022-04-07
1,2,Heath Hembree,PIT,0.00,18.00,0.00,0.00,,0.0%,40.0%,...,226,226,,0.00,9.11,-9.11,,9.11,13.13,2022-04-07


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
65,66,Aaron Fletcher,PIT,0.00,0.00,0.00,18.00,,0.0%,0.0%,...,794,225,,36.00,32.11,3.89,,9.07,6.35,2022-04-07
66,67,Spencer Strider,ATL,22.50,0.00,5.00,0.00,,83.3%,0.0%,...,-46,-27,,0.00,-1.89,1.89,,-1.15,-0.86,2022-04-07


2022-04-09
https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=1&ind=0&pageitems=2000000000&qual=0&team=0&startdate=2022-04-06&enddate=2022-04-08&month=1000&season1=2022&season=2022&sortcol=3&sortdir=desc&pagenum=1


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
0,1,Josh Staumont,KCR,0.00,27.00,0.00,0.00,,0.0%,50.0%,...,312,310,,0.00,12.11,-12.11,,12.11,16.56,2022-04-07
1,2,Heath Hembree,PIT,0.00,18.00,0.00,0.00,,0.0%,40.0%,...,226,226,,0.00,9.11,-9.11,,9.11,13.13,2022-04-07


...

,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
0,1,Josh Staumont,KCR,0.00,27.00,0.00,0.00,,0.0%,50.0%,...,312,310,,0.00,12.11,-12.11,,12.11,16.56,2022-04-07
1,2,Heath Hembree,PIT,0.00,18.00,0.00,0.00,,0.0%,40.0%,...,226,226,,0.00,9.11,-9.11,,9.11,13.13,2022-04-07


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
122166,864,Freddy Tarnok,ATL,13.50,0.00,1.00,0.00,,33.3%,0.0%,...,4,4,,0.00,0.11,-0.11,,0.11,1.02,2022-10-01
122167,865,Cam Alldred,PIT,9.00,0.00,1.00,0.00,,25.0%,0.0%,...,29,65,,0.00,1.11,-1.11,,2.59,2.05,2022-10-01


2022-10-03
https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=1&ind=0&pageitems=2000000000&qual=0&team=0&startdate=2022-04-06&enddate=2022-10-02&month=1000&season1=2022&season=2022&sortcol=3&sortdir=desc&pagenum=1


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
0,1,Josh Staumont,KCR,0.00,27.00,0.00,0.00,,0.0%,50.0%,...,312,310,,0.00,12.11,-12.11,,12.11,16.56,2022-04-07
1,2,Heath Hembree,PIT,0.00,18.00,0.00,0.00,,0.0%,40.0%,...,226,226,,0.00,9.11,-9.11,,9.11,13.13,2022-04-07


,#,Name,Team,K/9,BB/9,K/BB,HR/9,-,K%,BB%,...,FIP-,xFIP-,-,ERA,FIP,E-F,-,xFIP,SIERA,Stats Through This Date
123033,866,Freddy Tarnok,ATL,13.50,0.00,1.00,0.00,,33.3%,0.0%,...,4,4,,0.00,0.11,-0.11,,0.11,1.02,2022-10-02
123034,867,Cam Alldred,PIT,9.00,0.00,1.00,0.00,,25.0%,0.0%,...,29,65,,0.00,1.11,-1.11,,2.59,2.05,2022-10-02


In [15]:
pitcher_data.to_csv('daily_pitchers_data_2022.csv')